# signal.py

This notebook was automatically converted from a Python script.

# *SciPy Sinyal İşleme (scipy.signal)*


SciPy'ın signal modülü, sinyal işleme için çeşitli fonksiyonlar ve araçlar sunar. Bu modül, filtreleme, spektral analiz, sinyal tasarımı ve dönüşüm gibi işlemler için kullanılır.


Gerekli kütüphaneleri import edelim


In [ ]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt



## *1. Sinyal Oluşturma ve Görselleştirme*


İlk olarak, üzerinde çalışabileceğimiz bazı örnek sinyaller oluşturalım.


Zaman vektörü oluşturalım


In [ ]:
fs = 1000  # Örnekleme frekansı (Hz)
t = np.arange(0, 1, 1/fs)  # 1 saniyelik sinyal, 1000 Hz örnekleme frekansı



Farklı frekanslarda sinüs dalgaları oluşturalım


In [ ]:
f1, f2, f3 = 5, 50, 120  # Frekanslar (Hz)
x1 = np.sin(2 * np.pi * f1 * t)  # 5 Hz sinüs dalgası
x2 = 0.5 * np.sin(2 * np.pi * f2 * t)  # 50 Hz sinüs dalgası
x3 = 0.25 * np.sin(2 * np.pi * f3 * t)  # 120 Hz sinüs dalgası



Sinyalleri birleştirelim


In [ ]:
x = x1 + x2 + x3



Görselleştirelim


In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(t[:200], x1[:200], label='5 Hz')
plt.plot(t[:200], x2[:200], label='50 Hz')
plt.plot(t[:200], x3[:200], label='120 Hz')
plt.legend()
plt.title('Bileşen Sinyaller (ilk 200 örnek)')
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(t[:200], x[:200])
plt.title('Birleşik Sinyal (ilk 200 örnek)')
plt.grid(True)
plt.tight_layout()
plt.show()



## *2. Filtreler*


Sinyal işlemede filtreleme, istenmeyen frekans bileşenlerini bastırmak veya belirli frekans bileşenlerini vurgulamak için kullanılır. SciPy'ın signal modülü çeşitli filtre tasarım fonksiyonları sunar.


### *2.1. FIR Filtreler (Sonlu Dürtü Yanıtı)*


FIR filtre tasarımı ve uygulaması


Alçak geçiren FIR filtre (50 Hz altındaki frekansları geçiren)


In [ ]:
numtaps = 101  # Filtre uzunluğu
cutoff = 30  # Kesim frekansı (Hz)
fir_filter = signal.firwin(numtaps, cutoff, fs=fs)



Filtreyi uygulayalım


In [ ]:
filtered_signal = signal.lfilter(fir_filter, 1.0, x)



Sonuçları görselleştirelim


In [ ]:
plt.figure(figsize=(12, 8))



Filtre katsayıları


In [ ]:
plt.subplot(3, 1, 1)
plt.stem(np.arange(numtaps), fir_filter)
plt.title('FIR Filtre Katsayıları')
plt.grid(True)



Filtre frekans yanıtı


In [ ]:
w, h = signal.freqz(fir_filter, worN=8000)
plt.subplot(3, 1, 2)
plt.plot(0.5 * fs * w / np.pi, np.abs(h))
plt.axvline(cutoff, color='r', linestyle='--', alpha=0.7)
plt.xlabel('Frekans (Hz)')
plt.ylabel('Kazanç')
plt.title('Filtre Frekans Yanıtı')
plt.grid(True)
plt.xlim(0, 200)



Orijinal ve filtrelenmiş sinyal


In [ ]:
plt.subplot(3, 1, 3)
plt.plot(t[:200], x[:200], label='Orijinal Sinyal')
plt.plot(t[:200], filtered_signal[:200], label='Filtrelenmiş Sinyal')
plt.xlabel('Zaman (s)')
plt.title('Orijinal ve Filtrelenmiş Sinyal')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



### *2.2. IIR Filtreler (Sonsuz Dürtü Yanıtı)*


IIR filtre tasarımı


Butterworth band-pass filtre (20-80 Hz arasındaki frekansları geçiren)


In [ ]:
order = 4  # Filtre derecesi
lowcut = 20  # Alt kesim frekansı (Hz)
highcut = 80  # Üst kesim frekansı (Hz)



Normalize edilmiş kesim frekansları (Nyquist frekansına göre)


In [ ]:
nyq = 0.5 * fs
low = lowcut / nyq
high = highcut / nyq



Butterworth band-pass filtre tasarımı


In [ ]:
b, a = signal.butter(order, [low, high], btype='band')



Filtreyi uygulayalım


In [ ]:
filtered_signal_iir = signal.filtfilt(b, a, x)



Sonuçları görselleştirelim


In [ ]:
plt.figure(figsize=(12, 8))



Filtre frekans yanıtı


In [ ]:
w, h = signal.freqz(b, a, worN=8000)
plt.subplot(2, 1, 1)
plt.plot(0.5 * fs * w / np.pi, np.abs(h))
plt.axvline(lowcut, color='r', linestyle='--', alpha=0.7)
plt.axvline(highcut, color='r', linestyle='--', alpha=0.7)
plt.xlabel('Frekans (Hz)')
plt.ylabel('Kazanç')
plt.title('Butterworth Band-Pass Filtre Frekans Yanıtı')
plt.grid(True)
plt.xlim(0, 200)



Orijinal ve filtrelenmiş sinyal


In [ ]:
plt.subplot(2, 1, 2)
plt.plot(t[:200], x[:200], label='Orijinal Sinyal')
plt.plot(t[:200], filtered_signal_iir[:200], label='Filtrelenmiş Sinyal (IIR)')
plt.xlabel('Zaman (s)')
plt.title('Orijinal ve IIR Filtrelenmiş Sinyal')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()



## *3. Spektral Analiz*


Spektral analiz, sinyalin frekans bileşenlerini incelemek için kullanılır.


Hızlı Fourier Dönüşümü (FFT) ile spektral analiz


In [ ]:
X = np.fft.fft(x)
freqs = np.fft.fftfreq(len(x), 1/fs)



Pozitif frekansları alalım


In [ ]:
X_mag = np.abs(X[:len(x)//2]) / len(x) * 2  # Genlik spektrumu
freqs_pos = freqs[:len(x)//2]



Görselleştirelim


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(freqs_pos, X_mag)
plt.axvline(f1, color='r', linestyle='--', alpha=0.7, label=f'{f1} Hz')
plt.axvline(f2, color='g', linestyle='--', alpha=0.7, label=f'{f2} Hz')
plt.axvline(f3, color='b', linestyle='--', alpha=0.7, label=f'{f3} Hz')
plt.xlabel('Frekans (Hz)')
plt.ylabel('Genlik')
plt.title('Sinyal Spektrumu')
plt.xlim(0, 150)
plt.grid(True)
plt.legend()
plt.show()



### *3.1. Güç Spektral Yoğunluğu (Power Spectral Density)*


Güç spektral yoğunluğu, sinyalin frekans başına düşen gücünü gösterir.


Welch yöntemi ile güç spektral yoğunluğu hesaplama


In [ ]:
f, Pxx = signal.welch(x, fs, nperseg=1024)

plt.figure(figsize=(10, 6))
plt.semilogy(f, Pxx)
plt.axvline(f1, color='r', linestyle='--', alpha=0.7, label=f'{f1} Hz')
plt.axvline(f2, color='g', linestyle='--', alpha=0.7, label=f'{f2} Hz')
plt.axvline(f3, color='b', linestyle='--', alpha=0.7, label=f'{f3} Hz')
plt.xlabel('Frekans (Hz)')
plt.ylabel('PSD (V^2/Hz)')
plt.title('Güç Spektral Yoğunluğu')
plt.xlim(0, 150)
plt.grid(True)
plt.legend()
plt.show()



### *3.2. Spektrogram*


Spektrogram, sinyalin zaman-frekans temsilidir ve sinyalin frekans içeriğinin zamanla nasıl değiştiğini gösterir.


Farklı frekansları içeren yeni bir sinyal oluşturalım


In [ ]:
t_long = np.arange(0, 3, 1/fs)  # 3 saniyelik sinyal
chirp_signal = signal.chirp(t_long, f0=1, f1=150, t1=3, method='linear')



Gürültü ekleyelim


In [ ]:
np.random.seed(42)
noisy_chirp = chirp_signal + 0.2 * np.random.randn(len(t_long))



Spektrogram hesaplayalım


In [ ]:
f, t_spec, Sxx = signal.spectrogram(noisy_chirp, fs, nperseg=256, noverlap=128)



Görselleştirelim


In [ ]:
plt.figure(figsize=(12, 8))



Sinyal


In [ ]:
plt.subplot(2, 1, 1)
plt.plot(t_long, noisy_chirp)
plt.title('Chirp Sinyali (Doğrusal Frekans Artışı + Gürültü)')
plt.xlabel('Zaman (s)')
plt.grid(True)



Spektrogram


In [ ]:
plt.subplot(2, 1, 2)
plt.pcolormesh(t_spec, f, 10 * np.log10(Sxx), shading='gouraud', cmap='viridis')
plt.colorbar(label='Güç/Frekans (dB/Hz)')
plt.ylabel('Frekans (Hz)')
plt.xlabel('Zaman (s)')
plt.title('Spektrogram')
plt.tight_layout()
plt.show()



## *4. Sinyal İşleme Fonksiyonları*


### *4.1. Konvolüsyon*


Konvolüsyon, iki sinyalin etkileşimini modelleyen matematiksel bir işlemdir ve filtre uygulamanın temelidir.


Basit bir örnek: Kare dalga ve bir düşük geçiren filtre


Kare dalga sinyali oluşturalım


In [ ]:
t_square = np.linspace(0, 1, 1000, endpoint=False)
square_wave = signal.square(2 * np.pi * 5 * t_square)  # 5 Hz kare dalga



Basit bir düşük geçiren FIR filtre oluşturalım


In [ ]:
window = signal.windows.hann(51)  # Hanning penceresi
low_pass = signal.firwin(51, 10, fs=1000)  # 10 Hz kesim frekanslı alçak geçiren filtre



Konvolüsyon uygulayalım


In [ ]:
filtered_square = signal.convolve(square_wave, low_pass, mode='same')



Görselleştirelim


In [ ]:
plt.figure(figsize=(12, 8))



Kare dalga ve filtre


In [ ]:
plt.subplot(3, 1, 1)
plt.plot(t_square[:200], square_wave[:200])
plt.title('Kare Dalga Sinyali')
plt.grid(True)

plt.subplot(3, 1, 2)
plt.stem(np.arange(len(low_pass)), low_pass)
plt.title('Alçak Geçiren Filtre Katsayıları')
plt.grid(True)



Konvolüsyon sonucu


In [ ]:
plt.subplot(3, 1, 3)
plt.plot(t_square[:200], filtered_square[:200])
plt.title('Filtrelenmiş Sinyal (Konvolüsyon Sonucu)')
plt.xlabel('Zaman (s)')
plt.grid(True)

plt.tight_layout()
plt.show()



### *4.2. Korelasyon*


Korelasyon, iki sinyal arasındaki benzerliği ölçen bir işlemdir.


İki sinyal oluşturalım ve korelasyonlarını hesaplayalım


In [ ]:
t_corr = np.linspace(0, 1, 1000, endpoint=False)
sig1 = np.sin(2 * np.pi * 10 * t_corr)  # 10 Hz sinüs
sig2 = np.sin(2 * np.pi * 10 * t_corr + np.pi/4)  # 10 Hz sinüs, faz farkı ile



Korelasyon hesaplayalım


In [ ]:
correlation = signal.correlate(sig1, sig2, mode='full')
lags = signal.correlation_lags(len(sig1), len(sig2), mode='full')
lag_time = lags / 1000  # zaman birimine dönüştürme



Görselleştirelim


In [ ]:
plt.figure(figsize=(12, 8))



Orijinal sinyaller


In [ ]:
plt.subplot(2, 1, 1)
plt.plot(t_corr, sig1, label='Sinyal 1')
plt.plot(t_corr, sig2, label='Sinyal 2')
plt.title('Orijinal Sinyaller')
plt.legend()
plt.grid(True)



Korelasyon


In [ ]:
plt.subplot(2, 1, 2)
plt.plot(lag_time, correlation)
plt.axvline(0, color='r', linestyle='--')
plt.title('Korelasyon')
plt.xlabel('Zaman Gecikmesi (s)')
plt.grid(True)

plt.tight_layout()
plt.show()



### *4.3. Sinyal Pikleri (Tepe Noktaları) Bulma*


Pikler içeren bir sinyal oluşturalım


In [ ]:
t_peaks = np.linspace(0, 10, 1000)
x_peaks = np.sin(2 * np.pi * 0.5 * t_peaks) + 0.5 * np.sin(2 * np.pi * 2 * t_peaks) + 0.1 * np.random.randn(len(t_peaks))



Pikleri bulalım


In [ ]:
peaks, _ = signal.find_peaks(x_peaks, height=0.5, distance=50)



Görselleştirelim


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(t_peaks, x_peaks)
plt.plot(t_peaks[peaks], x_peaks[peaks], 'ro')
plt.title('Sinyal Pikleri')
plt.xlabel('Zaman')
plt.grid(True)
plt.show()



## *5. Sayısal Filtre Tasarımı*


SciPy, çeşitli filtre türlerini tasarlamak için kapsamlı araçlar sunar.


Farklı IIR filtre tiplerini karşılaştıralım


Butterworth, Chebyshev, Elliptik


Filtre parametreleri


In [ ]:
order = 5
cutoff = 100  # Hz
fs = 1000  # Hz
nyq = 0.5 * fs
normal_cutoff = cutoff / nyq



Farklı filtre türlerini tasarlayalım


In [ ]:
b_butter, a_butter = signal.butter(order, normal_cutoff, btype='low')
b_cheby1, a_cheby1 = signal.cheby1(order, 1, normal_cutoff, btype='low')  # 1 dB ripple
b_cheby2, a_cheby2 = signal.cheby2(order, 30, normal_cutoff, btype='low')  # 30 dB bastırma
b_ellip, a_ellip = signal.ellip(order, 1, 60, normal_cutoff, btype='low')  # 1 dB ripple, 60 dB bastırma



Frekans yanıtlarını hesaplayalım


In [ ]:
w, h_butter = signal.freqz(b_butter, a_butter, worN=2000)
w, h_cheby1 = signal.freqz(b_cheby1, a_cheby1, worN=2000)
w, h_cheby2 = signal.freqz(b_cheby2, a_cheby2, worN=2000)
w, h_ellip = signal.freqz(b_ellip, a_ellip, worN=2000)



Görselleştirelim


In [ ]:
plt.figure(figsize=(12, 8))



Lineer ölçekte


In [ ]:
plt.subplot(2, 1, 1)
plt.plot(0.5 * fs * w / np.pi, np.abs(h_butter), label='Butterworth')
plt.plot(0.5 * fs * w / np.pi, np.abs(h_cheby1), label='Chebyshev I')
plt.plot(0.5 * fs * w / np.pi, np.abs(h_cheby2), label='Chebyshev II')
plt.plot(0.5 * fs * w / np.pi, np.abs(h_ellip), label='Elliptik')
plt.axvline(cutoff, color='k', linestyle='--', alpha=0.5)
plt.xlabel('Frekans (Hz)')
plt.ylabel('Kazanç')
plt.title('Filtre Frekans Yanıtları (Lineer Ölçek)')
plt.grid(True)
plt.legend()
plt.xlim(0, 300)



Logaritmik ölçekte


In [ ]:
plt.subplot(2, 1, 2)
plt.plot(0.5 * fs * w / np.pi, 20 * np.log10(np.abs(h_butter)), label='Butterworth')
plt.plot(0.5 * fs * w / np.pi, 20 * np.log10(np.abs(h_cheby1)), label='Chebyshev I')
plt.plot(0.5 * fs * w / np.pi, 20 * np.log10(np.abs(h_cheby2)), label='Chebyshev II')
plt.plot(0.5 * fs * w / np.pi, 20 * np.log10(np.abs(h_ellip)), label='Elliptik')
plt.axvline(cutoff, color='k', linestyle='--', alpha=0.5)
plt.xlabel('Frekans (Hz)')
plt.ylabel('Kazanç (dB)')
plt.title('Filtre Frekans Yanıtları (Logaritmik Ölçek)')
plt.grid(True)
plt.legend()
plt.xlim(0, 300)
plt.ylim(-80, 5)

plt.tight_layout()
plt.show()



## *Sonuç*


SciPy'ın signal modülü, sinyal işleme için güçlü araçlar sağlar:


1. Filtreler (FIR ve IIR)


2. Spektral analiz (FFT, PSD, Spektrogram)


3. Konvolüsyon ve korelasyon


4. Sinyal özellikleri (pik tespiti, vb.)


5. Sayısal filtre tasarımı


Bu işlevler, bilimsel ve mühendislik uygulamalarında sinyal analizi ve işleme için temel araçlardır. 
